In [1]:
from scipy.integrate import ode
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, initializers
from sklearn.preprocessing import MinMaxScaler
from time import time
from os import path

In [2]:
from tensorflow.keras.layers import Dense, Dropout

#Ustvarimo funkcijo, ki vrača kompajlan model
def create_model(activation='sigmoid', optimizer='adam'):
    
    model = tf.keras.models.Sequential()
    model.add(Dense(3, input_dim=3, activation=activation))
    model.add(Dense(81, activation=activation))
    #model.add(Dropout(0.2))
    model.add(Dense(81, activation=activation))
    #model.add(Dropout(0.2))
    model.add(Dense(81, activation=activation))
    #model.add(Dropout(0.2))
    model.add(Dense(81, activation=activation))
    #model.add(Dropout(0.2))
    model.add(Dense(3, activation='sigmoid'))

    model.compile(loss='mean_absolute_error', optimizer=optimizer, metrics=['mae'])

    return model

In [3]:
from sklearn.model_selection import KFold

In [4]:
if(path.exists("data/data-2.npy") != 1):
    print("Generating data.")
    n = int(tmax/dt) + 1
    data = np.zeros((n, 3))

    i = 0
    start = time()
    while t < tmax:
        y_ATM = solver_ATM.integrate(t+dt)
        #y0_MOD = solver_MOD.integrate(t+dt)
        data[i, :] = y_ATM[:]
        i = i + 1
        t += dt
    print("Elasped time: %0.3g s" %(time()-start))
    data = data[1000:] # only use data from the attractor (after time = 10000 * dt)
    print("Integration finished.")
    np.save("data/data-2.npy", data)
    
else:
    print("Data exist.")
    data = np.load("data/data-2.npy")
    print("Data loaded.")
    
#PREPROCESS DATA
x_train = data[:-1,:]
y_train = data[1:, :] - data[:-1, :]

x_scaler=MinMaxScaler()
y_scaler=MinMaxScaler()

x_scaler.fit(x_train)
y_scaler.fit(y_train)

x_train_norm=x_scaler.transform(x_train)
y_train_norm=y_scaler.transform(y_train)

xy_train_norm=np.concatenate((x_train_norm,y_train_norm), axis=1)

new_order=np.random.choice(range(x_train.shape[0]),x_train.shape[0],replace=False)
x_train_norm=x_train_norm[new_order]
y_train_norm=y_train_norm[new_order]

Data exist.
Data loaded.


In [8]:
X = x_train_norm
Y = y_train_norm

num_folds = 3 # dolocimo st. foldov
kf = KFold(n_splits=num_folds, shuffle=True) # razred za razdelitev podatkov

fold_no = 1
scores_per_fold = []
verbose=2
losses = []

# ce nasa Loss ne vec pade se ustavi minimizacija
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=0, mode='min') 
#ce dosezemo plato zmanjsamo korak ucenja
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=5, min_lr=0.0001)

    
for train, test in kf.split(X[:len(X)], Y[:len(Y)]): #zanka cez vse usne ter testne sete
    modelCV = create_model()
    history = modelCV.fit(X[train], Y[train], epochs=100, batch_size=1024, verbose=verbose,
                          callbacks=[callback,reduce_lr])
    
    # na fitanem modelu izvrednotimo mere uspesnosti
    # mere uspesnosti bi lahko spremljali ze tekom ucenja, znotraj metode fit
    scores = modelCV.evaluate(X[test], Y[test], verbose=verbose)
    print(f'Score per fold {fold_no}: {modelCV.metrics_names} of {scores}')
    scores_per_fold.append(scores)
    
    losses.append(moving_average(history.history['loss'], 40))
    fold_no += 1

Epoch 1/100
651/651 - 2s - loss: 0.0338 - mae: 0.0338
Epoch 2/100
651/651 - 2s - loss: 0.0323 - mae: 0.0323
Epoch 3/100
651/651 - 2s - loss: 0.0284 - mae: 0.0284
Epoch 4/100
651/651 - 2s - loss: 0.0270 - mae: 0.0270
Epoch 5/100
651/651 - 2s - loss: 0.0250 - mae: 0.0250
Epoch 6/100
651/651 - 2s - loss: 0.0218 - mae: 0.0218
Epoch 7/100
651/651 - 2s - loss: 0.0209 - mae: 0.0209
Epoch 8/100
651/651 - 2s - loss: 0.0204 - mae: 0.0204
Epoch 9/100
651/651 - 2s - loss: 0.0196 - mae: 0.0196
Epoch 10/100
651/651 - 2s - loss: 0.0177 - mae: 0.0177
Epoch 11/100
651/651 - 2s - loss: 0.0150 - mae: 0.0150
Epoch 12/100
651/651 - 2s - loss: 0.0123 - mae: 0.0123
Epoch 13/100


KeyboardInterrupt: 